# Stock capital gain calculation
## First in First out law (FIFO)

### - Import libraries

In [ ]:
import pandas as pd

### - read excel files

In [ ]:
df = pd.read_excel("tax_2021.xlsx", sheet_name="Sheet2")
df.head()

### Extract required data 

In [ ]:
df_1 = pd.read_excel("tax_2021.xlsx", sheet_name="Sheet2", skiprows=1, nrows=9)
df_1.drop(columns=['Company', 'Unit Price ($)', 'Trade Value ($)', 'Brokerage+GST ($)', 'GST ($)', 'Contract Note'], inplace=True)
df_1['Unit Value ($)'] = df_1['Total Value ($)'] / df_1['Quantity']

df_1

In [ ]:
df_2= pd.read_excel("tax_2021.xlsx", sheet_name="Sheet2", skiprows=12, nrows=4)
df_2.drop(columns=['Company', 'Unit Price ($)', 'Trade Value ($)', 'Brokerage+GST ($)', 'GST ($)', 'Contract Note'], inplace=True)
df_2['Unit Value ($)'] = df_2['Total Value ($)'] / df_2['Quantity']
df_2

In [ ]:
df_3= pd.read_excel("tax_2021.xlsx", sheet_name="Sheet2", skiprows=21, nrows=3)
df_3.drop(columns=['Company', 'Unit Price ($)', 'Trade Value ($)', 'Brokerage+GST ($)', 'GST ($)', 'Contract Note'], inplace=True)
df_3['Unit Value ($)'] = df_3['Total Value ($)'] / df_3['Quantity']
df_3

In [ ]:
df_4= pd.read_excel("tax_2021.xlsx", sheet_name="Sheet2", skiprows=27, nrows=17)
df_4.drop(columns=['Company', 'Unit Price ($)', 'Trade Value ($)', 'Brokerage+GST ($)', 'GST ($)', 'Contract Note'], inplace=True)
df_4['Unit Value ($)'] = df_4['Total Value ($)'] / df_4['Quantity']
df_4

## Concatenate 2020-2021 Data 

In [ ]:
df = pd.concat([df_1, df_2, df_3, df_4], axis=0) # Concatenate 2020 and 2021 data into a table
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True) # Date : String to datetime format
df.sort_values(['Code', 'Date'], ascending=True, inplace=True) # Sorting multiple names [Code, Date]
df.reset_index(drop=True, inplace=True) # Reset index from 0
df

## Divide the dataframe by Stock Code

In [ ]:
mask1 = df['Code'] == 'MQG'
mask2 = df['Code'] == 'OSH'
mask3 = df['Code'] == 'IOZ'
mask4 = df['Code'] == 'NDQ'
mask5 = df['Code'] == 'SYI'
mask6 = df['Code'] == 'ETHI'

In [ ]:
df1 = df[mask1]
df1.reset_index(drop=True, inplace=True)
df1

In [ ]:
df2 = df[mask2]
df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
df3 = df[mask3]
df3.reset_index(drop=True, inplace=True)
df3

In [ ]:
df4 = df[mask4]
df4.reset_index(drop=True, inplace=True)
df4

In [ ]:
df5 = df[mask5]
df5.reset_index(drop=True, inplace=True)
df5

In [ ]:
df6 = df[mask6]
df6.reset_index(drop=True, inplace=True)
df6

## Gain caluculation

In [ ]:
import logging
from collections import deque
import math

class Trans:
    datetime=None
    amount=None
    price=None
 
    def __init__(self, datetime, amount, price):
        self.datetime=datetime
        self.amount=amount
        self.price=price
     
    def getInfo(self):
        return(str(self.datetime)+"; "+
                str(self.amount)+"; "+
                str(self.price))+"; "
 
def balanceFifo(all_trans):
 
    qTransactions = deque() 
    trans_result = list()
 
    for t in all_trans:
        #Add first element to the queue
        if len(qTransactions)==0:
            #logging.debug('Added the first element: %s',t.getInfo())
            qTransactions.append(t)
            continue
 
        while (t.amount!=0 and len(qTransactions)>0):
            #investigate the first element from the queue
            tq=qTransactions.popleft()
            #the same type of transaction: both sell or both buy
            if tq.amount*t.amount>0:
                #return the first element back to the same place
                qTransactions.appendleft(tq)
                #add the new element to the list
                qTransactions.append(t)
                #logging.debug('Added: %s',t.getInfo())
                break
             
            #contrary transactions: (sell and buy) or (buy and sell) 
            if tq.amount*t.amount<0:
                #logging.debug('Transaction : %s',t.getInfo())
                #logging.debug('... try to balance with: %s',tq.getInfo())
 
                #The element in the queue have more units and takes in the current transaction
                if abs(tq.amount)>abs(t.amount):
                    result = insertTransaction(tq.datetime,t.datetime,\
                            math.copysign(t.amount,tq.amount), tq.price,t.price)
                    trans_result.append(result)
                    
                    #update the amount of the element in the queue
                    tq.amount=tq.amount+t.amount
                    #return the element back to the same place
                    qTransactions.appendleft(tq)
                    #logging.debug('Removed transaction: %s',t.getInfo())
                    #the transaction has been balanced, take a new transaction
                    break
                 
                #The element from the queue and transaction have the same amount of units
                if abs(tq.amount)==abs(t.amount):
                    result = insertTransaction(tq.datetime,t.datetime,\
                                math.copysign(t.amount,tq.amount), tq.price,t.price)
                    trans_result.append(result)
                    
                    #update the amount in the transaction 
                    t.amount=0
                    #logging.debug('Balanced, removed transaction: %s',t.getInfo())
                    #logging.debug('Balanced, removed from the queue: %s',tq.getInfo())
                    #the transaction has been balanced, take a new transaction
                    continue
                    
                #The transaction has more units
                if abs(tq.amount)<abs(t.amount):
                    #update the units in transaction, (remove element from the queue)
                    t.amount=t.amount+tq.amount
                    result = insertTransaction(tq.datetime,t.datetime,tq.amount,tq.price,t.price)
                    trans_result.append(result)
                    #logging.debug('Removed from queue: %s',tq.getInfo())
                     
                    #the transaction has not been balanced, 
                    #take a new element from the queue (t.amount>0)
                    continue
                 
        #We have unbalanced transaction but the queue is empty            
        if (t.amount!=0 and len(qTransactions)==0):
            #Add unbalanced transaction to the queue
            #The queue changes polarisation
            qTransactions.append(t)
            #logging.debug('Left element: %s',t.getInfo())
     
     
    #If something remained in the queue, treat it as open or part-open transactions
    while (len(qTransactions)>0):
        tq=qTransactions.popleft()
        #logging.debug('Remained on list transaction: %s',tq.getInfo())
        
    return trans_result
 
def insertTransaction(dateStart,dateEnd,amount,priceStart,priceEnd):
    #print("Bought={}, sold={},  amount={}, buy price={}, sell_price={}, gain={}".\
    #        format(dateStart,dateEnd,amount,priceStart,priceEnd, amount*(priceEnd-priceStart)))
    result = [dateStart,dateEnd,amount,priceStart,priceEnd, amount*(priceEnd-priceStart)]
    return result

### Gain Calculation for each Stock Code

In [ ]:
trans_list=list()
for i in range(0, df1.shape[0]):
    trans = Trans(df1['Date'][i].date(), df1['Quantity'][i], df1['Unit Value ($)'][i])
    trans_list.append(trans)
trans_result = balanceFifo(trans_list)
df1_capital = pd.DataFrame(trans_result, columns = ['Date Purchased', 'Date Sold', 'Quantity', 'Buy Price', 'Sell Price', 'Capital Gain'])
df1_capital

In [ ]:
print(df1['Code'][0], df1_capital['Capital Gain'].sum())

In [ ]:
trans_list=list()
for i in range(0, df2.shape[0]):
    trans = Trans(df2['Date'][i].date(), df2['Quantity'][i], df2['Unit Value ($)'][i])
    trans_list.append(trans)
trans_result = balanceFifo(trans_list)
df2_capital = pd.DataFrame(trans_result, columns = ['Date Purchased', 'Date Sold', 'Quantity', 'Buy Price', 'Sell Price', 'Capital Gain'])
df2_capital

In [ ]:
print(df2['Code'][0], df2_capital['Capital Gain'].sum())

In [ ]:
trans_list=list()
for i in range(0, df3.shape[0]):
    trans = Trans(df3['Date'][i].date(), df3['Quantity'][i], df3['Unit Value ($)'][i])
    trans_list.append(trans)
trans_result = balanceFifo(trans_list)
df3_capital = pd.DataFrame(trans_result, columns = ['Date Purchased', 'Date Sold', 'Quantity', 'Buy Price', 'Sell Price', 'Capital Gain'])
df3_capital

In [ ]:
trans_list=list()
for i in range(0, df4.shape[0]):
    trans = Trans(df4['Date'][i].date(), df4['Quantity'][i], df4['Unit Value ($)'][i])
    trans_list.append(trans)
trans_result = balanceFifo(trans_list)
df4_capital = pd.DataFrame(trans_result, columns = ['Date Purchased', 'Date Sold', 'Quantity', 'Buy Price', 'Sell Price', 'Capital Gain'])
df4_capital

In [ ]:
trans_list=list()
for i in range(0, df5.shape[0]):
    trans = Trans(df5['Date'][i].date(), df5['Quantity'][i], df5['Unit Value ($)'][i])
    trans_list.append(trans)
trans_result = balanceFifo(trans_list)
df5_capital = pd.DataFrame(trans_result, columns = ['Date Purchased', 'Date Sold', 'Quantity', 'Buy Price', 'Sell Price', 'Capital Gain'])
df5_capital

In [ ]:
print(df5['Code'][0], df5_capital['Capital Gain'].sum())

In [ ]:
trans_list=list()
for i in range(0, df6.shape[0]):
    trans = Trans(df6['Date'][i].date(), df6['Quantity'][i], df6['Unit Value ($)'][i])
    trans_list.append(trans)
trans_result = balanceFifo(trans_list)
df6_capital = pd.DataFrame(trans_result, columns = ['Date Purchased', 'Date Sold', 'Quantity', 'Buy Price', 'Sell Price', 'Capital Gain'])
df6_capital